In [ ]:
import pandas as pd
from connection import connect
from utils.model_loader import ModelRegistry
from utils.translate_language import convert_language


In [ ]:
#connection
SCHEMA='production'
co_oltp,etl_conn=connect(SCHEMA)

In [ ]:
# Carga de modelos
registry = ModelRegistry()
registry.preload_model('en', 'es')
registry.preload_model('en', 'fr')
tokenizer_es, model_es = registry.get_model('en', 'es')
tokenizer_fr, model_fr = registry.get_model('en', 'fr')

## Extract

In [ ]:
# extract DimProductCategory
category=pd.read_sql_table('product_category',co_oltp,schema=SCHEMA)
category


## Transform


In [ ]:
category=category.rename(columns={'product_category_id':'product_category_alternate_key','name':'english_product_category_name'})

category=category.drop(['rowguid','modified_date'],axis=1)

category=convert_language('english_product_category_name','french_product_category_name', tokenizer_fr, model_fr, category)
category=convert_language('english_product_category_name','spanish_product_category_name', tokenizer_es, model_es,category)

category

## Load

In [ ]:
category.to_sql('dim_product_category', etl_conn, if_exists='append',index=False)